## Evaluacion de modelos

#### El siguiente script hace la evaluacion de un conjunto de imagenes haciendo uso de un modelo. A continuacion se importan las librerias necesarias para su ejecución

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix

import time
from tqdm.auto import tqdm

### Valores de entrada

#### La variable test_folder indica la carpeta en donde se encuentran las imagenes que se quieren a evaluar, esta carpeta debe tener dos subcarpetas una Fake y otra Real que corresponden a la naturaleza de la imagen.

#### La variable model hace referencia al modelo previamente entrenado que hace el proceso de clasificación

In [ ]:
test_folder = '/home/icesi/Documents/DeepFakeFinal/Experimento_8/'
model = '/home/icesi/Documents/DeepFakeFinal/Experimento_7_Xception_Rostros_FineTune/Model/model_fine_final.h5'
classes = 'classes.txt'

In [ ]:
predictions = []
partial_filenames = [] 
backup_batch_size = 500

for dirname, _, filenames in os.walk(test_folder + 'Fake/'):
    pbar = tqdm(total=len(filenames))
    for filename in filenames:
        command_result = !python inference.py {model} {classes} {os.path.join(dirname, filename)} --top_n 2
        probability = command_result[len(command_result)-1]
        partial_filenames.append(filename)
        predictions.append(probability) 
        if(len(partial_filenames) % backup_batch_size == 0):
            result_fake_df = pd.DataFrame({"filename": partial_filenames, "probability": predictions})
            result_fake_df.to_csv(test_folder+"result_fake.csv", index=False)
        pbar.update(1)  
        
result_fake_df = pd.DataFrame({"filename": filenames, "probability": predictions})
result_fake_df.to_csv(test_folder+"result_fake.csv", index=False)

pbar.close()

predictions = []
partial_filenames = [] 
backup_batch_size = 500
for dirname, _, filenames in os.walk(test_folder + 'Real/'):
    pbar = tqdm(total=len(filenames))
    for filename in filenames:
        command_result = !python inference.py {model} {classes} {os.path.join(dirname, filename)} --top_n 2
        probability = command_result[len(command_result)-1]
        partial_filenames.append(filename)
        predictions.append(probability)
        if(len(partial_filenames) % backup_batch_size == 0):
            result_fake_df = pd.DataFrame({"filename": partial_filenames, "probability": predictions})
            result_fake_df.to_csv(test_folder+"result_real.csv", index=False)
        pbar.update(1)  
        
result_fake_df = pd.DataFrame({"filename": filenames, "probability": predictions})
result_fake_df.to_csv(test_folder+"result_real.csv", index=False)

pbar.close()

## Evaluación

#### En esta seccíon se generan las metricas de evaluación de acuerdo a la evaluacion de cada imagen.

In [ ]:
data_fake = pd.read_csv(test_folder+"result_fake.csv", sep=",")
data_real = pd.read_csv(test_folder+"result_real.csv", sep=",")
data_fake['ImageReal']=1
data_real['ImageReal']=0
data=[data_real,data_fake]
data=pd.concat(data)
data['Result']=0
data

In [ ]:
def optimization():
    i=1
    value=0.00
    bestI=1
    while(i<100):
        data["Result"] = np.where((data["probability"]>i),1, data["Result"])
        data["Result"] = np.where((data["probability"]<=i),0, data["Result"])
        y_true = np.array(data['ImageReal'])
        y_scores = np.array(data['Result'])
        v=roc_auc_score(y_true, y_scores)
        if(value<v):
            value=v
            bestI=i
        i+=1
    return bestI

In [ ]:
indice=optimization()
data["Result"] = np.where((data["probability"]>indice),1, data["Result"])
data["Result"] = np.where((data["probability"]<=indice),0, data["Result"])
data.info()
print("Best Indice = "+str(indice))

In [ ]:
y_true = np.array(data['ImageReal'])
y_scores = np.array(data['Result'])
roc_auc_score(y_true, y_scores)

In [ ]:
print('Confusion Matrix :')
print(confusion_matrix(y_true, y_scores))
print ('Accuracy Score :',accuracy_score(y_true, y_scores) )
print( 'Report : ')
print (classification_report(y_true, y_scores) )